In [1]:
import numpy as np
import pandas as pd
import joblib
import datetime
import matplotlib.pyplot as plt
from hyperopt import hp, Trials, STATUS_OK, fmin, anneal
from functools import partial
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [2]:
# Load the datasets
train_df = pd.read_excel("train_data.xlsx")
test_df = pd.read_excel("test_data.xlsx")

In [3]:
# Define feature matrix and target variable
X_train = train_df.drop(['P'], axis=1)
y_train = train_df['P']
X_test = test_df.drop(['P'], axis=1)
y_test = test_df['P']

In [4]:
# Define Model Evaluation Metric
def model_metrics(model, X, y):
    """Evaluate model performance using RMSE."""
    y_pred = model.predict(X)
    return np.sqrt(mean_squared_error(y, y_pred))

In [5]:

# Bayesian Optimization Function
def bayes_fmin(X_train, X_test, y_train, y_test, eval_iters=100):
    """Optimize hyperparameters using Bayesian optimization."""
    
    def objective(params):
        model = GradientBoostingRegressor(
            learning_rate=float(params['learning_rate']),
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            max_features=int(params['max_features']),
            random_state=42
        )
        model.fit(X_train, y_train)
        loss = model_metrics(model, X_test, y_test)
        
        with open('./GBR-Bayesian-fitness-RMSE.txt', 'a+') as f:
            f.write(f"{loss}\n")
        
        return {"loss": loss, "status": STATUS_OK}
    
    space = {
        'n_estimators': hp.quniform('n_estimators', 1, 200, 1),
        'max_depth': hp.quniform('max_depth', 1, 50, 1),
        'learning_rate': hp.quniform('learning_rate', 0.0001, 1, 0.01),
        'max_features': hp.quniform('max_features', 1, 11, 1)
    }
    
    best_params = fmin(
        fn=objective,
        space=space,
        algo=partial(anneal.suggest),
        max_evals=eval_iters,
        trials=Trials(),
        return_argmin=True
    )
    
    return {k: int(v) if k != 'learning_rate' else float(v) for k, v in best_params.items()}

In [6]:
# Optimize Parameters
best_params = bayes_fmin(X_train, X_test, y_train, y_test, 500)
print("Optimized Parameters:", best_params)

# Train Final Model
gbr_reg = GradientBoostingRegressor(**best_params, random_state=42)
gbr_reg.fit(X_train, y_train)
y_train_pred, y_test_pred = gbr_reg.predict(X_train), gbr_reg.predict(X_test)

# Evaluate Model
metrics = {
    "Train RMSE": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "Train MAE": mean_absolute_error(y_train, y_train_pred),
    "Train R2": r2_score(y_train, y_train_pred),
    "Test RMSE": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "Test MAE": mean_absolute_error(y_test, y_test_pred),
    "Test R2": r2_score(y_test, y_test_pred)
}

for metric, value in metrics.items():
    print(f"INFO: {metric} = {value:.2f}")

100%|██████████| 500/500 [15:20<00:00,  1.84s/trial, best loss: 16.496310280593306]
Optimized Parameters: {'learning_rate': 0.1, 'max_depth': 6, 'max_features': 6, 'n_estimators': 99}
INFO: Train RMSE = 3.28
INFO: Train MAE = 2.33
INFO: Train R2 = 1.00
INFO: Test RMSE = 16.50
INFO: Test MAE = 7.13
INFO: Test R2 = 0.99
